# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import dask


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt


In [2]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

!pip install kaggle

In [4]:
!kaggle competitions DAT300

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2576            True  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      11305            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4654           False  
imagenet-object-localization-challenge         2029-12-31 07:00:00  Research         Knowledge         51           False  
competitive-data-science-predict-future-sales  2019-12-31 23:59:00  Playground           Kudos       4322           False  
Kannada-MNIST                                  2019-12-17 23:59:00  Playground       Knowledge        276           False  
bigquery

In [ ]:
def import_data(csv_file):
    """ Returns median of data.

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    df = dd.read_csv(csv_file)
    return df

In [ ]:
def inspect_missing_values(data):
    """ Counting missing values in a DataFrame
    
    :param data: pandas data frame
    :return: pandas data frame
    
    """
    missing_values = df.isnull().sum()
    return missing_values

In [ ]:
def percent_missing_values(data):
    """ Calculate the percent of missing values in the DataFrame
    
    :param data: pandas data frame
    :return: pandas data frame
    
    """
    missing_count = ((missing_values / df.index.size) * 100)
    return missing_count

Running computations with the compute method

In [ ]:
with ProgressBar():
    missing_count_pct = missing_count.compute()
missing_count_pct